# Data Collection for Europe Base Port Container Price Prediction

This notebook focuses on gathering all the raw data needed for our 1-week ahead container price forecasting project. We will collect data from three different sources and save it for later processing.

## What are we predicting?

**Target**: Europe Base Port container prices (1 week ahead)

**Base Ports Definition**: Average shipping cost for a 40-foot container from Shanghai/China to major European ports including Rotterdam (Netherlands), Hamburg (Germany), London (UK), and Antwerp (Belgium).

**Why these ports**: These represent the main entry points for Asian goods into Europe and provide a standard benchmark for European route pricing.

## Our data sources:

1. Shanghai Containerized Freight Index (local CSV file) - Main price data
2. Crude oil prices (EIA DCOILWTICO dataset) - Cost factor affecting shipping fuel costs
3. Geopolitical disruption data (from GDELT via BigQuery) - Black swan event indicators

**Note**: We use GDELT data exported from Google BigQuery for historical coverage (2018-2025). This provides weekly disruption metrics including conflict events, severe incidents, and sentiment analysis for shipping-critical regions.

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 1: Load the Shanghai Containerized Freight Index

In [4]:
try:
    # Load the raw CSV file
    # header=1 means the actual column names are in the second row (row 1, counting from 0)
    raw_df = pd.read_csv('data/Shanghai_Containerized_Freight_Index.csv', header=1)
    print(f"Successfully loaded the CSV file with {raw_df.shape[0]} rows and {raw_df.shape[1]} columns")
    
    # Display the first few rows to see what the data looks like
    print("\nFirst 5 rows of the raw data:")
    print(raw_df.head())
    
except FileNotFoundError:
    print("Error: The file 'data/Shanghai_Containerized_Freight_Index.csv' was not found.")
    print("Please make sure it is in the data folder.")

Successfully loaded the CSV file with 385 rows and 5 columns

First 5 rows of the raw data:
  the period (YYYY-MM-DD)  Comprehensive Index  Europe (Base port)  \
0                1/5/2018               816.58                 888   
1               1/12/2018               839.72                 897   
2               1/19/2018               840.36                 891   
3               1/26/2018               858.60                 907   
4                2/2/2018               883.59                 912   

   Mediterranean (Base port)  Persian Gulf and Red Sea (Dubai)  
0                        738                               433  
1                        759                               450  
2                        761                               572  
3                        772                               631  
4                        797                               611  


## Step 2: Select and rename the columns we need

The dataset has many columns for different routes, but we only need a few for our Europe Base Port prediction. We will select the date column, the overall freight index (SCFI), and the Europe Base Port price.

In [5]:
# Select only the columns we need
df_freight = raw_df[['the period (YYYY-MM-DD)', 'Comprehensive Index', 'Europe (Base port)']].copy()

# Rename columns to simpler names
df_freight.columns = ['Date', 'SCFI_Index', 'Europe_Base_Price']

print("Selected and renamed columns:")
print(df_freight.columns.tolist())
print(f"\nDataset now has {df_freight.shape[0]} rows and {df_freight.shape[1]} columns")
print("\nFirst 5 rows:")
print(df_freight.head())

Selected and renamed columns:
['Date', 'SCFI_Index', 'Europe_Base_Price']

Dataset now has 385 rows and 3 columns

First 5 rows:
        Date  SCFI_Index  Europe_Base_Price
0   1/5/2018      816.58                888
1  1/12/2018      839.72                897
2  1/19/2018      840.36                891
3  1/26/2018      858.60                907
4   2/2/2018      883.59                912


## Step 3: Convert date strings to proper date format

Right now, the Date column is just text. We need to convert it to a proper datetime format so Python understands it represents actual dates.

In [6]:
df_freight['Date'] = pd.to_datetime(df_freight['Date'], format='%m/%d/%Y', errors='coerce')

# Check how many dates were successfully converted
valid_dates = df_freight['Date'].notna().sum()
total_rows = len(df_freight)
print(f"Successfully converted {valid_dates} out of {total_rows} dates")

# Remove rows where date conversion failed
df_freight.dropna(subset=['Date'], inplace=True)
print(f"After removing invalid dates: {len(df_freight)} rows remaining")

# Set the Date column as the index (the row identifier)
df_freight.set_index('Date', inplace=True)

print("\nDate conversion complete!")
print(df_freight.head())

Successfully converted 385 out of 385 dates
After removing invalid dates: 385 rows remaining

Date conversion complete!
            SCFI_Index  Europe_Base_Price
Date                                     
2018-01-05      816.58                888
2018-01-12      839.72                897
2018-01-19      840.36                891
2018-01-26      858.60                907
2018-02-02      883.59                912


## Step 4: Convert price columns to numbers

Sometimes data is read as text even when it represents numbers. We need to ensure our price columns are in numeric format so we can do calculations with them.

In [7]:
# Convert price columns to numeric format
for col in ['SCFI_Index', 'Europe_Base_Price']:
    df_freight[col] = pd.to_numeric(df_freight[col], errors='coerce')
    print(f"Converted {col} to numeric type")

# Remove any rows with missing values
# This ensures we have complete data for all rows
before_drop = len(df_freight)
df_freight.dropna(inplace=True)
after_drop = len(df_freight)

print(f"\nRemoved {before_drop - after_drop} rows with missing values")
print(f"Final freight dataset: {after_drop} rows")
print(f"Date range: {df_freight.index.min().strftime('%Y-%m-%d')} to {df_freight.index.max().strftime('%Y-%m-%d')}")

print("\nFinal cleaned freight data:")
print(df_freight.head())

Converted SCFI_Index to numeric type
Converted Europe_Base_Price to numeric type

Removed 0 rows with missing values
Final freight dataset: 385 rows
Date range: 2018-01-05 to 2025-08-22

Final cleaned freight data:
            SCFI_Index  Europe_Base_Price
Date                                     
2018-01-05      816.58                888
2018-01-12      839.72                897
2018-01-19      840.36                891
2018-01-26      858.60                907
2018-02-02      883.59                912


## Step 5: Load crude oil price data

Oil prices affect shipping costs since ships use fuel. We will load historical crude oil prices from both EIA WTI and Brent EU datasets.

### Data Sources: EIA DCOILWTICO and DCOILBRENTEU

We use two major oil benchmarks from the U.S. Energy Information Administration (EIA):
- **DCOILWTICO**: WTI (West Texas Intermediate) crude oil prices
- **DCOILBRENTEU**: Brent EU crude oil prices

**Files**: `data/DCOILWTICO.csv` and `data/DCOILBRENTEU.csv`
**Format**: CSV with columns `observation_date` and price column
**Frequency**: Daily prices

### Why Both Oil Benchmarks?

WTI and Brent are the two major global oil benchmarks. WTI is more US-focused while Brent is more representative of global oil prices. Including both provides a more comprehensive view of fuel cost fluctuations affecting shipping.

### Processing Steps:
1. Load both CSV files
2. Convert dates to datetime format
3. Rename columns for consistency
4. Filter to match freight data date range
5. Handle missing values (marked as empty strings in EIA data)

In [8]:
print("=" * 70)
print("LOADING CRUDE OIL PRICE DATA")
print("=" * 70)

# Get the date range from our freight data
start_date = df_freight.index.min().strftime('%Y-%m-%d')
end_date = df_freight.index.max().strftime('%Y-%m-%d')

print(f"\nRequesting data from {start_date} to {end_date}")

df_oil_wti = pd.DataFrame()
df_oil_brent = pd.DataFrame()

# Load WTI crude oil prices from EIA DCOILWTICO dataset
print("\n--- Loading WTI from EIA DCOILWTICO.csv ---")

try:
    # Load the EIA WTI crude oil price data
    wti_file_path = 'data/DCOILWTICO.csv'
    df_oil_wti_raw = pd.read_csv(wti_file_path, parse_dates=['observation_date'], index_col='observation_date')

    # Rename the price column for consistency
    df_oil_wti_raw = df_oil_wti_raw.rename(columns={'DCOILWTICO': 'WTI_Price'})

    # Filter to our date range
    df_oil_wti = df_oil_wti_raw[(df_oil_wti_raw.index >= start_date) & (df_oil_wti_raw.index <= end_date)].copy()

    # Handle missing values (EIA uses empty strings for missing data)
    df_oil_wti['WTI_Price'] = pd.to_numeric(df_oil_wti['WTI_Price'], errors='coerce')

    # Remove rows with missing prices
    before_clean = len(df_oil_wti)
    df_oil_wti.dropna(subset=['WTI_Price'], inplace=True)
    after_clean = len(df_oil_wti)

    if not df_oil_wti.empty:
        print(f"Success: Loaded {len(df_oil_wti)} days of WTI crude oil price data from {wti_file_path}")
        print(f"  Date range: {df_oil_wti.index.min().strftime('%Y-%m-%d')} to {df_oil_wti.index.max().strftime('%Y-%m-%d')}")
        print(f"  Price range: ${df_oil_wti['WTI_Price'].min():.2f} to ${df_oil_wti['WTI_Price'].max():.2f}")
        print(f"  Average price: ${df_oil_wti['WTI_Price'].mean():.2f}")
        print(f"  Removed {before_clean - after_clean} rows with missing values")
        print("\nFirst 5 rows of WTI data:")
        print(df_oil_wti.head())
    else:
        print(f"Error: No valid WTI oil price data found in date range")

except FileNotFoundError:
    print(f"Error: The file '{wti_file_path}' was not found.")
    print("Please ensure the DCOILWTICO.csv file is in the data folder.")
except Exception as e:
    print(f"Error: An error occurred while loading WTI data: {str(e)[:150]}")

# Load Brent EU crude oil prices from EIA DCOILBRENTEU dataset
print("\n--- Loading Brent EU from EIA DCOILBRENTEU.csv ---")

try:
    # Load the EIA Brent EU crude oil price data
    brent_file_path = 'data/DCOILBRENTEU.csv'
    df_oil_brent_raw = pd.read_csv(brent_file_path, parse_dates=['observation_date'], index_col='observation_date')

    # Rename the price column for consistency
    df_oil_brent_raw = df_oil_brent_raw.rename(columns={'DCOILBRENTEU': 'Brent_Price'})

    # Filter to our date range
    df_oil_brent = df_oil_brent_raw[(df_oil_brent_raw.index >= start_date) & (df_oil_brent_raw.index <= end_date)].copy()

    # Handle missing values (EIA uses empty strings for missing data)
    df_oil_brent['Brent_Price'] = pd.to_numeric(df_oil_brent['Brent_Price'], errors='coerce')

    # Remove rows with missing prices
    before_clean = len(df_oil_brent)
    df_oil_brent.dropna(subset=['Brent_Price'], inplace=True)
    after_clean = len(df_oil_brent)

    if not df_oil_brent.empty:
        print(f"Success: Loaded {len(df_oil_brent)} days of Brent EU crude oil price data from {brent_file_path}")
        print(f"  Date range: {df_oil_brent.index.min().strftime('%Y-%m-%d')} to {df_oil_brent.index.max().strftime('%Y-%m-%d')}")
        print(f"  Price range: ${df_oil_brent['Brent_Price'].min():.2f} to ${df_oil_brent['Brent_Price'].max():.2f}")
        print(f"  Average price: ${df_oil_brent['Brent_Price'].mean():.2f}")
        print(f"  Removed {before_clean - after_clean} rows with missing values")
        print("\nFirst 5 rows of Brent EU data:")
        print(df_oil_brent.head())
    else:
        print(f"Error: No valid Brent EU oil price data found in date range")

except FileNotFoundError:
    print(f"Error: The file '{brent_file_path}' was not found.")
    print("Please ensure the DCOILBRENTEU.csv file is in the data folder.")
except Exception as e:
    print(f"Error: An error occurred while loading Brent EU data: {str(e)[:150]}")

# Fallback: Create synthetic oil price data if loading failed (last resort)
if df_oil_wti.empty and df_oil_brent.empty:
    print("\n--- Fallback: Creating synthetic placeholder data ---")
    print("Warning: No real oil data available. Creating synthetic data for demonstration.")
    print("This should only be used for testing. For production, obtain the DCOILWTICO.csv and DCOILBRENTEU.csv files.")

    # Create date range matching freight data
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    # Create synthetic oil prices with realistic values and volatility
    # Base price around $70-80 with random walk
    np.random.seed(42)  # For reproducibility
    base_price = 75.0
    random_walk = np.random.randn(len(date_range)).cumsum() * 2  # Random walk with std=2
    synthetic_prices = base_price + random_walk

    # Clip to reasonable range (50-120)
    synthetic_prices = np.clip(synthetic_prices, 50, 120)

    df_oil_wti = pd.DataFrame({
        'WTI_Price': synthetic_prices
    }, index=date_range)
    
    df_oil_brent = pd.DataFrame({
        'Brent_Price': synthetic_prices + np.random.randn(len(date_range)) * 2  # Slight variation
    }, index=date_range)

    print(f"Success: Created {len(df_oil_wti)} days of synthetic WTI oil price data")
    print(f"Success: Created {len(df_oil_brent)} days of synthetic Brent EU oil price data")
    print(f"  Date range: {df_oil_wti.index.min().strftime('%Y-%m-%d')} to {df_oil_wti.index.max().strftime('%Y-%m-%d')}")
    print(f"  WTI Price range: ${df_oil_wti['WTI_Price'].min():.2f} to ${df_oil_wti['WTI_Price'].max():.2f}")
    print(f"  Brent Price range: ${df_oil_brent['Brent_Price'].min():.2f} to ${df_oil_brent['Brent_Price'].max():.2f}")
    print(f"  WTI Average: ${df_oil_wti['WTI_Price'].mean():.2f}")
    print(f"  Brent Average: ${df_oil_brent['Brent_Price'].mean():.2f}")
    print("\nWarning: This is SYNTHETIC data. Replace with real DCOILWTICO.csv and DCOILBRENTEU.csv data for actual predictions.")

print("\n" + "=" * 70)
print("OIL DATA LOADING COMPLETE")
print("=" * 70)
print(f"Total WTI days: {len(df_oil_wti)}")
print(f"Total Brent EU days: {len(df_oil_brent)}")
print(f"Data sources: {'EIA DCOILWTICO & DCOILBRENTEU' if not df_oil_wti.empty and not df_oil_brent.empty else 'Synthetic'}")
print("=" * 70)

LOADING CRUDE OIL PRICE DATA

Requesting data from 2018-01-05 to 2025-08-22

--- Loading WTI from EIA DCOILWTICO.csv ---
Success: Loaded 1872 days of WTI crude oil price data from data/DCOILWTICO.csv
  Date range: 2018-01-05 to 2025-07-02
  Price range: $-36.98 to $123.64
  Average price: $68.34
  Removed 82 rows with missing values

First 5 rows of WTI data:
                  WTI_Price
observation_date           
2018-01-05            61.49
2018-01-08            61.73
2018-01-09            62.92
2018-01-10            63.60
2018-01-11            63.81

--- Loading Brent EU from EIA DCOILBRENTEU.csv ---
Success: Loaded 399 days of Brent EU crude oil price data from data/DCOILBRENTEU.csv
  Date range: 2018-01-05 to 2025-08-22
  Price range: $15.87 to $127.44
  Average price: $73.04
  Removed 0 rows with missing values

First 5 rows of Brent EU data:
                  Brent_Price
observation_date             
2018-01-05              68.01
2018-01-12              69.64
2018-01-19          

## Step 6: Load the new "black swan" geopolitical disruption data

We now load the new, richer BigQuery dataset that contains specific geopolitical and black swan event metrics for shipping-critical regions.

In [ ]:
print("=" * 70)
print("LOADING NEW BLACK SWAN GEOPOLITICAL DISRUPTION DATA")
print("=" * 70)

# Helper function to convert ISO year/week to Friday date
def iso_to_friday_date(row):
    """Converts ISO year and week to the Friday of that week."""
    return datetime.fromisocalendar(int(row['iso_year']), int(row['week']), 5)

# Load the new BigQuery export
try:
    news_file_path = 'data/bq-results-20251021-090045-1761037274833.csv'
    df_news_raw = pd.read_csv(news_file_path)
    print(f"Success: Loaded {len(df_news_raw)} weekly records from {news_file_path}")
    print(f"  Columns: {df_news_raw.columns.tolist()}")

    # Remove week 53 entries as they cause invalid week errors
    before_filter = len(df_news_raw)
    df_news_raw = df_news_raw[df_news_raw['week'] != 53]
    after_filter = len(df_news_raw)
    if before_filter != after_filter:
        print(f"Removed {before_filter - after_filter} week 53 entries (invalid weeks)")

    # Convert ISO week to Friday date to match freight data
    print("\nConverting ISO week numbers to Friday dates...")
    df_news_raw['date'] = df_news_raw.apply(iso_to_friday_date, axis=1)
    df_news_raw['date'] = pd.to_datetime(df_news_raw['date'])
    df_news_raw.set_index('date', inplace=True)
    print(f"Success: Converted dates. New range: {df_news_raw.index.min().date()} to {df_news_raw.index.max().date()}")

    # Drop the non-predictive global_worst_event_impact column
    if 'global_worst_event_impact' in df_news_raw.columns:
        df_news_raw = df_news_raw.drop(columns=['global_worst_event_impact'])
        print("Success: Dropped 'global_worst_event_impact' column.")

    # Save the cleaned data
    df_news_raw.to_csv('collected_news_data.csv')
    print(f"\nSuccess: New black swan data saved to 'collected_news_data.csv'")
    print(f"   Contains {len(df_news_raw.columns)} predictive features.")
    print("\nSample of new event data:")
    print(df_news_raw.head())

except FileNotFoundError:
    print(f"Error: The file '{news_file_path}' was not found.")
    print("Please ensure the BigQuery export CSV file is in the data folder.")
except Exception as e:
    print(f"An error occurred: {e}")

## Step 7: Save the collected data

We will save all three datasets as CSV files so we can use them in the next notebooks without having to fetch the data again.

In [9]:
# Save freight data
df_freight.to_csv('collected_freight_data.csv')
print("Saved freight data to 'collected_freight_data.csv'")
print(f"  {len(df_freight)} weekly records")

# Save WTI oil data
if not df_oil_wti.empty:
    df_oil_wti.to_csv('collected_wti_oil_data.csv')
    print("Saved WTI crude oil price data to 'collected_wti_oil_data.csv'")
    print(f"  {len(df_oil_wti)} daily records")
else:
    print("No WTI oil data to save")

# Save Brent EU oil data
if not df_oil_brent.empty:
    df_oil_brent.to_csv('collected_brent_oil_data.csv')
    print("Saved Brent EU crude oil price data to 'collected_brent_oil_data.csv'")
    print(f"  {len(df_oil_brent)} daily records")
else:
    print("No Brent EU oil data to save")

print("\n" + "=" * 70)
print("DATA COLLECTION COMPLETE!")
print("=" * 70)
print("\nSummary of collected data:")
print(f"Available: Freight data: {len(df_freight)} weeks ({df_freight.index.min().strftime('%Y-%m-%d')} to {df_freight.index.max().strftime('%Y-%m-%d')})")
print(f"Available: WTI oil data: {len(df_oil_wti) if not df_oil_wti.empty else 0} days")
print(f"Available: Brent EU oil data: {len(df_oil_brent) if not df_oil_brent.empty else 0} days")

# Black swan disruption data may not exist if the file failed to load; guard against NameError
if 'df_news_raw' in globals():
    try:
        print(f"Available: Black swan disruption data: {len(df_news_raw)} weeks")
        print(f"  Features: {', '.join(df_news_raw.columns.tolist())}")
    except Exception:
        # If df_news_raw exists but isn't a dataframe or has issues, report its type
        print(f"df_news_raw exists but could not be summarized (type={type(df_news_raw)})")
else:
    print("Available: Black swan disruption data: 0 weeks (df_news_raw not found)")

print("\nReady for Step 2: Data Understanding and Feature Engineering")
print("=" * 70)

Saved freight data to 'collected_freight_data.csv'
  385 weekly records
Saved WTI crude oil price data to 'collected_wti_oil_data.csv'
  1872 daily records
Saved Brent EU crude oil price data to 'collected_brent_oil_data.csv'
  399 daily records

DATA COLLECTION COMPLETE!

Summary of collected data:
Available: Freight data: 385 weeks (2018-01-05 to 2025-08-22)
Available: WTI oil data: 1872 days
Available: Brent EU oil data: 399 days
Available: Black swan disruption data: 0 weeks (df_news_raw not found)

Ready for Step 2: Data Understanding and Feature Engineering
